## In this exercise, we will use the layer model which we built last week. Here we will explore different optimizer functions . In the previous section, we implemented the SGD with momentum. It does this by adding a fraction of the update vector of the past time step to the current update vector which accelerates the parameter updates for faster convergence 

### Nesterov momentum

The momentum is about the rolling the ball from hill following the slope, Nesterov momentum is little smart which calculates the gradient based on the future state of parameter instead of current state

In [2]:
class Nesterov:
    def __init__(self, learning_rate = 0.01, momentum=0.9):
        self.learning_rate = learning_rate 
        self.momentum = momentum
        self.w_updated = None
    
    def update(self, w, grad):
        #calculate the gradient of future
        
        approx_future_grad =  self.momentum
        
        if self.w_updated is None:
            self.w_updated = np.zeros(np.shape(w))        
        
        #use the momentum if any
        self.w_updated = (self.momentum * self.w_updated) + (1-self.momentum) * grad        
        return w - (self.learning_rate * grad) 

In [3]:
#to be continued